# TFY4345 Computational assignment 1
_Thorvald Molthe Ballestad_

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from matplotlib import rc
# Set common figure parameters:
newparams = {'axes.labelsize': 11, 'axes.linewidth': 1, 'savefig.dpi': 300, 
             'lines.linewidth': 1.0, 'figure.figsize': (20, 10),
             'ytick.labelsize': 10, 'xtick.labelsize': 10,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,}
plt.rcParams.update(newparams)

We will investigate a simple pendulum limited to small oscillations, so that we have an harmonic oscillator.

In [ ]:
# Setup
m = 1 #kg
R = 1 #m
g = 9.81 #m/s^2

theta_0 = 0.2 # rad, inital angle
omega_0 = 0.0 # rad/s, inital angular velocity

end_time = 7 # s, the time at which the simulation will end

The problem consists of two first-order differential equations:
$$
\dot{\theta} = \omega\\
\dot{\omega} = \frac{F_{\perp}}{mR}.
$$

Let 
$$
y = 
\begin{bmatrix}
\theta\\
\omega
\end{bmatrix},
$$
then
$$
y' = 
\begin{bmatrix}
y_2\\
-\sin(y_1) / mR
\end{bmatrix}
= f(y).
$$

We will do the euler step:
$$
y(t+dt) = y(t) + f(y(t)) dt
$$

In [ ]:
def f(y):
    """f(y) = y'"""
    return np.array([y[1], -g*np.sin(y[0])/R])

In [ ]:
def euler_step(y, f, dt):
    """Returns an euler-approximation of y(t+dt) given y(t)"""
    return y + f(y)*dt

In [ ]:
def euler_solve(y, f, dt):
    """Euler's method
    Parameters:
        y  : ndarray, with size (num_iterations, dim_of_problem), to fill with computed values.
             y[0] must be initial values
        f  : function that represents dy/dt
        dt : timestep to use
    Returns:
        y, with same size as input y, filled with solution"""
    for i in range(len(y)- 1):
        y[i+1] = euler_step(y[i], f, dt)
    return y

def euler_cromer_solve(y, f, dt):
    """Euler-Cromer's method
    Parameters:
        y  : ndarray, with size (num_iterations, dim_of_problem), to fill with computed values.
             y[0] must be initial values
        f  : function that represents dy/dt
        dt : timestep to use
    Returns:
        y, with same size as input y, filled with solution"""
    for i in range(len(y) - 1):
        theta, omega = euler_step(y[i], f, dt)
        theta = y[i,0] + omega*dt
        y[i+1] = np.array([theta, omega])
    return y

def rk4_solve(y, f, dt):
    """Runge-Kutta 4
    Parameters:
        y  : ndarray, with size (num_iterations, dim_of_problem), to fill with computed values.
             y[0] must be initial values
        f  : function that represents dy/dt
        dt : timestep to use
    Returns:
        y, with same size as input y, filled with solution"""
    for i in range(len(y) - 1):
        k_1 = f(y[i])*dt
        k_2 = f(y[i] + k_1/2)*dt
        k_3 = f(y[i] + k_2/2)*dt
        k_4 = f(y[i] + k_3)*dt
        y[i+1] = y[i] + (k_1 + 2*k_2 + 2*k_3 + k_4)/6
    return y

In [ ]:
def kinetic_energy(omega):
    return 0.5*m*R**2*omega**2

def potential_energy(theta):
    return m*g*R*(1-np.cos(theta))

def total_energy(theta, omega):
    return kinetic_energy(omega) + potential_energy(theta)

def plt_method_energy(t, theta, omega, name):
    plt.plot(t, potential_energy(theta), label=f"{name} potential")
    plt.plot(t, kinetic_energy(theta), label=f"{name} kinetic")
    plt.plot(t, total_energy(theta, omega), label=f"{name} total")

## Part one, Euler method for different time-delta

In [ ]:
# Using Euler's method

def do_method(end_time, dt, solver=euler_solve, theta_0=theta_0, omega_0=omega_0):
    """Use method given end_time, dt
    Returns:
     t: array of times
     y: 2D-array with [theta, omega]"""
    num_iterations = int(np.ceil(end_time/dt))
    y = np.zeros((num_iterations, 2))
    y[0] = np.array([theta_0, omega_0])
    y = solver(y, f, dt)
    t = np.arange(0, end_time, dt)
    
    return t, y

def analytical(t, theta_0=theta_0, omega_0=omega_0):
    """Analytical solution to small-angle approx. (ie. harm. osc.)"""
    omega = np.sqrt(g/R) # Angular frequency
    A = np.sqrt(theta_0**2 + (omega_0/omega)**2)
    phi = np.arctan(-omega_0/(omega*theta_0))
    return A*np.sin(omega*t + phi + np.pi/2)

# Euler's method for different time deltas
dt_list = [1e-4, 1e-3, 5e-3, 1e-2, 5e-2]
y_euler_dt = {}
for dt in dt_list:
    y_euler_dt[dt] = do_method(end_time, dt)

plt.suptitle("Position and energy as a function of time for different dime deltas",
                fontsize=24, y=1.1)
for i in range(len(dt_list)):
    plt.subplot(len(dt_list), 2, 2*i+1)
    dt = dt_list[i]
    plt.title(f"dt = {dt}")
    t = y_euler_dt[dt][0]
    y = y_euler_dt[dt][1]
    plt.plot(t, y[:,0], label="Numerical")
    t_n = np.linspace(0, end_time, 100)
    plt.plot(t_n, analytical(t_n), label="Analytical")
    plt.xlabel("$\theta$")
    plt.legend(loc="lower left")
    
    plt.subplot(len(dt_list), 2, 2*i+2)
    plt_method_energy(t, y[:,0], y[:,1], "")
    plt.legend(loc="lower left")
    
plt.tight_layout()

We begin to see a serious deviation from energy conservation at dt=5e-3, dt=1e-3 should be the maximum time step, depending on how many cycles one wishes to simulate one might have to go lower.

## Part two, more sophisticated methods

In [ ]:
dt = 0.05

y_dict = {}
method_list = [euler_solve, euler_cromer_solve, rk4_solve]
for method in method_list:
    t, y_dict[method.__repr__()] = do_method(end_time, dt, method)

## Let's do some plotting!!
First is $\omega(t)$ and energy for all methods

In [ ]:
plt.title("$\\theta(t)$")
for method in method_list:
    y = y_dict[method.__repr__()]
    method_name = method.__doc__.split('\n')[0]
    theta = y[:,0]
    omega = y[:,1]
    plt.plot(t, theta, label=method_name)
    plt.xlabel("t [s]")
    plt.ylabel("$\\theta(t)$ [rad]")
    plt.legend()
    plt.show()
    
    plt_method_energy(t, theta, omega, method_name)
    plt.xlabel("t [s]")
    plt.ylabel("$E$ [J]")
    plt.legend()
    plt.show()

Plotted in phase space:

In [ ]:
for method in method_list:
    y = y_dict[method.__repr__()]
    theta = y[:,0]
    omega = y[:,1]
    plt.plot(theta, omega, label=method.__doc__.split('\n')[0])
    plt.xlabel("$\\theta$")
    plt.ylabel("$\omega$")
    plt.legend()
    plt.show()

### Varying time step for Euler-Cromer and RK4

In [ ]:
dt_list = [1e-3, 1e-2, 1e-1, 3e-1, 5e-1, 1]
for dt in dt_list:
    t, y_euler_cromer = do_method(end_time, dt, euler_cromer_solve)
    _, y_rk4 = do_method(end_time, dt, rk4_solve)
    theta_euler_cromer, omega_euler_cromer = y_euler_cromer[:,0], y_euler_cromer[:,1]
    theta_rk4, omega_rk4 = y_rk4[:,0], y_rk4[:,1]
    plt.subplot("211")
    plt.title(f"dt = {dt}")
    plt.plot(theta_euler_cromer, omega_euler_cromer, label="Euler-Cromer")
    plt.plot(theta_rk4, omega_rk4, label="Runge-Kutta 4")
    plt.axhline(y=0)
    plt.axvline(x=0)
    plt.xlabel("$\\theta$")
    plt.ylabel("$\omega$")
    plt.legend()
    plt.subplot("212")
    plt.ylabel("$E[j]$")
    plt.xlabel("$t[s]$")
    plt.plot(t, total_energy(theta_euler_cromer, omega_euler_cromer), label="Euler-Cromer")
    plt.plot(t, total_energy(theta_rk4, omega_rk4), label="Runge-Kutta 4")
    plt.legend()
    plt.show()

We wish to plot the error in energy as a function of dt, in order to see where it "diverges". There are several ways to do this, for example plotting $\Delta E = E(t_{final}) - E(t_0)$. However, since the energy for Euler-Cromer oscillates(without diverging) even at very small dt, this method might be affected by change, since it is random where on the oscillation $E(t_{final})$ is.

An alternative approach is to look at the square integral of E. Even with the oscillations, we expect this value to remain constant(but non-zero), until the dt that gives divergence, where the integral will diverge.

In [ ]:
N = 40
#dt_list = np.geomspace(0.2, 1, N) # Use geomspace (equally spaced in log-space), instead of linspace
dt_list = np.linspace(0.2, 1, N)
energy_loss_ec = np.zeros(N)
energy_loss_e = np.zeros(N)

method_list = [euler_cromer_solve, rk4_solve]
energy_loss = np.zeros((N, len(method_list)))
for i in range(N):
    dt = dt_list[i]
    for j, method in enumerate(method_list):
        t, y = do_method(end_time, dt, method)
        theta = y[:,0]
        omega = y[:,1]
        engy = total_energy(theta, omega)
        energy_loss[i, j] = np.sum(np.square(engy-engy[0])) # The -engy[0], sets E(t_0) as zero
        
        # Different possible metrics for divergence:
        # Integral of E
        ## engy = total_energy(theta, omega)
        ## energy_loss[i, j] = np.sum(engy-engy[0])
        
        # E(t_final) - E(t_0)
        ## engy = total_energy(theta[[0, -1]], omega[[0, -1]]) 
        ## energy_loss[i,j] = engy[-1] - engy[0]
        
        # <E_final> - E(t_0), where <E_final> is averaged over some (3) final points
        ## engy = total_energy(theta[[0, -3, -2, -1]], omega[[0, -3, -2, -1]])
        ## energy_loss[i,j] = np.average(engy[[-3, -2, -1]]) - engy[0]
 


plt.title("Energy loss, E(t=t_end) - E(t=0), as a function of dt")
plt.ylim([-5, 5])
plt.axhline(y=0, color='grey')
for i, method in enumerate(method_list):
    plt.plot(dt_list, energy_loss[:, i], label=method.__doc__.split('\n')[0])
plt.legend()
plt.show()

We se that Euler-Cromer diveres at about dt=0.5 and RK4 at about dt=0.9